In [3]:
pip install opencv-python numpy matplotlib torch torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
from cv2 import imshow as cv2_imshow
import numpy
import math
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import os

In [5]:
def otsu(img):
  img = np.array(img)
  pxls = img.flatten()
  img_dict = {}
  for i in sorted(pxls):
    if i not in img_dict:
      img_dict[i] = 1
    else:
      img_dict[i] +=1

  N = 0
  for i in img_dict.values():
    N += i
  density = {}
  for i in img_dict.keys():
    density[i] = img_dict[i]/N
  mean_l = {}
  weight_l = {}
  sum = 0
  w = 0
  for i in range(len(density)):
    w += density[list(density.keys())[i]]
    
    weight_l[list(density.keys())[i]] = w
    sum += density[list(density.keys())[i]] * np.sum(pxls)
    numer = 0
    for j in range(i + 1):
      numer += density[list(density.keys())[j]]* np.sum(pxls) * list(density.keys())[j]
    mean_l[list(density.keys())[i]] = numer/sum
  mean_r = {}
  dens_l = [(i, density[i]) for i in density]
  for i in range(len(dens_l)):
    numer = 0
    sum = 0
    if(i < len(dens_l) - 1):
      for j in range(i + 1, len(dens_l)):
        numer += dens_l[j][0]*dens_l[j][1]*np.sum(pxls)
        sum += dens_l[j][1]*np.sum(pxls)
    if(dens_l[i][0] != dens_l[len(dens_l)-1][0]):
      mean_r[dens_l[i][0]] = numer/sum
    else:
      mean_r[dens_l[i][0]] = 0
  vars = {}
  for i in mean_l:
    var = (w-weight_l[i])*weight_l[i]*((mean_l[i] - mean_r[i])**2)
    vars[i] = var
  threshold = sorted(vars, key = vars.get)[-1]
  for i in range(len(img)):
    for j in range(len(img[i])):
      if(img[i][j] <= threshold):
        img[i][j] = 0
      else:
        img[i][j] = 255
  return img

In [6]:
signum = lambda x: -1 if x < 0 else 1

cells = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def get_angle(left, right):
    angle = left - right
    if abs(angle) > 180:
        angle = -1 * signum(angle) * (360 - abs(angle))
    return angle

In [7]:
def poincare_index_at(i, j, angles, tolerance):
    deg_angles = [math.degrees(angles[i - k][j - l]) % 180 for k, l in cells]
    index = 0
    for k in range(0, 8):
        if abs(get_angle(deg_angles[k], deg_angles[k + 1])) > 90:
            deg_angles[k + 1] += 180
        index += get_angle(deg_angles[k], deg_angles[k + 1])

    if 180 - tolerance <= index and index <= 180 + tolerance:
        return "loop"
    if -180 - tolerance <= index and index <= -180 + tolerance:
        return "delta"
    if 360 - tolerance <= index and index <= 360 + tolerance:
        return "whorl"
    return "none"

In [8]:
def calculate_singularities(im, angles, tolerance, W):
    (y, x) = im.shape
    
    #result = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    im = cv2.cvtColor(im, cv2.COLOR_GRAY2RGB)

    cores = []
    for i in range(1, len(angles) - 1):
        for j in range(1, len(angles[i]) - 1):
            singularity = poincare_index_at(i, j, angles, tolerance)
            if singularity != "none":
                if(singularity!="delta"):
                    cores.append((i*W, j*W))
    return cores


In [9]:
def get_kernel(image, center):
    (x, y) = center
    
    kernel = []
    for i in range(max(0, x-8), min(x+8, 239)):
        temp = []
        for j in range(max(0, y - 8), min(y+8, 319)):
            temp.append(list(image[i][j]))
        kernel.append(temp)
    return kernel

In [10]:
def apply_kernel_at(get_value, kernel, i, j):
    kernel_size = 3
    result = 0
    for k in range(0, kernel_size):
        for l in range(0, kernel_size):
            pixel = get_value(i + k - kernel_size / 2, j + l - kernel_size / 2)
            result += pixel * kernel[k][l]
    return result

In [11]:
def transpose(ls):
    return map(list, zip(*ls))

In [12]:
def calculate_angles(im, W, f, g):
    (y, x) = im.shape
    im_load = im
    get_pixel = lambda x, y: im_load[int(y)][int(x)]
    
    sobelOperator = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    
    ySobel = list(sobelOperator)
    xSobel = list(transpose(sobelOperator))

    result = [[] for i in range(1, x, W)]
    G = [[] for i in range(1, x, W)]

    for i in range(1, x, W):
        for j in range(1, y, W):
            nominator = 0
            denominator = 0
            for k in range(i, min(i + W , x - 1)):
                for l in range(j, min(j + W, y - 1)):
                    Gx = apply_kernel_at(get_pixel, xSobel, k, l)
                    Gy = apply_kernel_at(get_pixel, ySobel, k, l)

                    nominator += f(Gx, Gy)
                    denominator += g(Gx, Gy)
            angle = (math.pi + math.atan2(nominator, denominator)) / 2
            result[(i - 1) // W].append(angle)

    return result

In [13]:
def draw_elipse(image, center, color):
  (x, y) = center
  cv2.ellipse(image, (x, y), (16, 16), 0, 0, 360, color, 2)

In [14]:
def poincare_indices(url):
  img = cv2.imread(url, cv2.COLOR_BGR2GRAY)
  blockSize = 24
  f = lambda x, y: 2 * x * y
  g = lambda x, y: x ** 2 - y ** 2
  img = otsu(img)

  angles = calculate_angles(img, blockSize, f, g)
  img_t = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
  colors = {"loop" : (255, 0, 0), "delta" : (0, 255, 0), "whorl": (0, 0, 255)}
  cores = calculate_singularities(img, angles, 1, blockSize)
  return cores, img.shape


In [15]:
def add_to_dataset(img, core, count):
    label_path = r'C:\Users\Arsalan\Desktop\CoresResearch\PoreGroundTruthMarked'
    img = img[:-4]
    with open(label_path + "\\" + str(count)+".txt", 'a') as file1:
        file1.write(f"{core[0]} {core[1]}\n")


In [16]:

image_path = r'./Fingerprints'
path = image_path
label_path = r'./Labels'

fig = plt.figure(figsize=(30, 30))
rows = 6
columns = 5

pics = os.listdir(path)
count = 1
for img in pics:

    if(count==31):
        break
    img_p = path + '\\' + img
    print(img_p)
    try:
        cores, shape = poincare_indices(img_p)
        coord= [int(numpy.mean([i[0] for i in cores])), int(numpy.mean([i[1] for i in cores]))]
        
        def draw_circle(event,x,y,flags,param):
            #global mouseX,mouseY
            if event == cv2.EVENT_LBUTTONDBLCLK:
                cv2.circle(img,(x,y),100,(255,0,0),-1)
                print(x, y)

        img_cv2 = cv2.imread(img_p)
        img_cv2 = cv2.ellipse(img_cv2, coord, (24, 24), 0, 0, 360, (0, 0, 255), 1)
        cv2.imshow('Correct?', img_cv2)
        cv2.setMouseCallback('image', draw_circle)
        cv2.waitKey()
        cv2.destroyAllWindows()

        if(input("Do you want to add this picture")=='y'):
            fig.add_subplot(rows,columns, count)

            plt.imshow(img_cv2)
            plt.axis('off')
            plt.title(f'img: {count}.bmp')
                
            print(coord, f'{count}.bmp')
            cv2.imwrite(image_path + "\\" + str(count) +'.bmp', cv2.imread(img_p, cv2.COLOR_BGR2GRAY))
            with open(label_path + "\\" + str(count)+".txt", 'a') as file1:
                file1.write(f"{coord[0]} {coord[1]}\n")

                count += 1
    
    except:
        print(img)
    

./Fingerprints\1.bmp
1.bmp
./Fingerprints\10.bmp
10.bmp
./Fingerprints\11.bmp
11.bmp
./Fingerprints\12.bmp


In [ ]:
imgs = os.listdir(r'C:\Users\Arsalan\Desktop\Research\Fingerprints')
print(len(imgs))